In [36]:
required_packages <- c(
  "forecast",       # ARIMAX modeling
  "dplyr",          # Data manipulation
  "tidyr",          # pivot_wider/pivot_longer
  "prophet",        # xreg forecasting
  "thief",          # h
  "TSrepr",         # Errors
  "progress",
  "doParallel",
  "foreach",
  "vars",
  "lmtest",
  "tseries"
)

# Install missing packages
new_packages <- required_packages[!required_packages %in% installed.packages()[,"Package"]]
if(length(new_packages)) install.packages(new_packages)

# Load all packages
invisible(lapply(required_packages, library, character.only = TRUE))

In [37]:
sales <- read.csv("sales_train_validation.csv", stringsAsFactors = FALSE)
calendar <- read.csv("calendar.csv", stringsAsFactors = FALSE)

In [38]:
calendar <- calendar %>%
  mutate(
    date = as.Date(date),
    is_holiday = 0  # Initialize all as 0 (not holidays)
  )

# Loop through each row (day) and check if it has any holiday-related events
for (i in 1:nrow(calendar)) {
  if (calendar$event_name_1[i] != "" | calendar$event_type_1[i] != "" |
      calendar$event_name_2[i] != "" | calendar$event_type_2[i] != "") {
    calendar$is_holiday[i] <- 1  # Set is_holiday to 1 if there's any event
  }
}

In [39]:
sales <- sales %>% mutate(row_id = row_number())
item_metadata <- sales %>% select(row_id, dept_id)
sales_long <- sales %>%
  select(row_id, starts_with("d_")) %>%
  pivot_longer(cols = starts_with("d_"), names_to = "day", values_to = "value")

ERROR: Error in select(., row_id, dept_id): unused arguments (row_id, dept_id)


In [ ]:
dept_sales <- sales_long %>%
  left_join(calendar %>% select(day = d, date), by = "day") %>%
  left_join(item_metadata, by = "row_id") %>%
  group_by(date, dept_id) %>%
  summarise(dept_sales = sum(value, na.rm = TRUE), .groups = "drop") %>%
  pivot_wider(
    names_from = dept_id,
    values_from = dept_sales,
    values_fill = 0
  ) %>%
  arrange(date)

In [ ]:
sales_with_categories <- sales %>%
  mutate(
    row_id = row_number(),
    category = case_when(
      grepl("^FOODS", item_id) ~ "FOODS",
      grepl("^HOBBIES", item_id) ~ "HOBBIES",
      grepl("^HOUSEHOLD", item_id) ~ "HOUSEHOLD",
      TRUE ~ "OTHER"
    )
  ) %>%
  filter(category != "OTHER")

In [ ]:
category_sales <- sales_with_categories %>%
  select(row_id, category, starts_with("d_")) %>%
  pivot_longer(cols = starts_with("d_"), names_to = "day", values_to = "value")

In [ ]:
category_sales_aggregated <- category_sales %>%
  left_join(calendar %>% select(day = d, date), by = "day") %>%
  group_by(date, category) %>%
  summarise(category_sales = sum(value, na.rm = TRUE), .groups = "drop") %>%
  pivot_wider(
    names_from = category,
    values_from = category_sales,
    values_fill = 0
  ) %>%
  arrange(date)

In [ ]:
# Future regressors (from external file)
future_regressors <- read.csv("prognozes_isplestos_platus.csv")

In [ ]:
last_train_date <- as.Date("2016-04-24")  # Last day of training data in M5
first_forecast_date <- last_train_date + 1

In [ ]:
level9_sales <- sales %>%
  group_by(store_id, dept_id, cat_id) %>%
  summarise(across(starts_with("d_"), sum), .groups = "drop") %>%
  pivot_longer(starts_with("d_"), names_to = "day", values_to = "sales") %>%
  left_join(calendar %>% select(day = d, date), by = "day") %>%
  filter(date <= last_train_date)  # Critical: only training period

In [40]:
x1 = level9_sales %>%
    filter(store_id == "CA_1",
           dept_id == "FOODS_1") %>%
    arrange(date)
adf.test(ts(x1$sales))

x2 = level9_sales %>%
    filter(store_id == "TX_2",
           dept_id == "HOUSEHOLD_2") %>%
    arrange(date)
adf.test(ts(x2$sales))

Warning message in adf.test(ts(x1$sales)):
“p-value smaller than printed p-value”



	Augmented Dickey-Fuller Test

data:  ts(x1$sales)
Dickey-Fuller = -6.4361, Lag order = 12, p-value = 0.01
alternative hypothesis: stationary


Warning message in adf.test(ts(x2$sales)):
“p-value smaller than printed p-value”



	Augmented Dickey-Fuller Test

data:  ts(x2$sales)
Dickey-Fuller = -5.4176, Lag order = 12, p-value = 0.01
alternative hypothesis: stationary


In [29]:
historical_regressors <- calendar %>%
  filter(date <= last_train_date) %>%  # Only historical dates
  select(date, is_holiday) %>%
  left_join(
    level9_sales %>%
      group_by(date, dept_id) %>%
      summarise(dept_sales = sum(sales), .groups = "drop") %>%
      pivot_wider(names_from = dept_id, values_from = dept_sales),
    by = "date"
  ) %>%
  left_join(
    level9_sales %>%
      group_by(date, cat_id) %>%
      summarise(cat_sales = sum(sales), .groups = "drop") %>%
      pivot_wider(names_from = cat_id, values_from = cat_sales),
    by = "date"
  ) %>%
  replace(is.na(.), 0)

ERROR: Error in select(., date, is_holiday): unused arguments (date, is_holiday)


In [43]:
forecast_state_item <- function(item_data, store_id, dept_id, cat_id) {
  ts_data <- ts(item_data$sales, frequency = 7)
  
  # Historical regressors (training period only)
  xreg_hist <- historical_regressors %>%
    filter(date %in% item_data$date) %>%
    select(is_holiday, all_of(dept_id), all_of(cat_id)) %>% #select(is_holiday, all_of(dept_id), all_of(cat_id)) %>%
    as.matrix()
  
  # Future regressors (forecast period only)
  xreg_future <- future_regressors %>%
    select(is_holiday, all_of(dept_id), all_of(cat_id)) %>% #select(is_holiday, all_of(dept_id), all_of(cat_id)) %>%
    as.matrix()
  
  # Model fitting and forecasting (same as before)
  fit <- tryCatch({
    auto.arima(ts_data, xreg = xreg_hist, lambda = 0, biasadj = TRUE, seasonal = TRUE,
               stepwise = TRUE, approximation = TRUE)
  }, error = function(e) {
    Arima(ts_data, order = c(1,1,1), seasonal = c(0,1,1), xreg = xreg_hist)
  })
  
  fc <- if ("xreg" %in% names(fit$call)) {
    forecast(fit, h = 28, xreg = xreg_future)
  } else {
    forecast(fit, h = 28)
  }
  
  data.frame(
    store_id = store_id,
    dept_id = dept_id,
    date = seq(max(item_data$date) + 1, length.out = 28, by = "day"),
    forecast = as.numeric(fc$mean),
    stringsAsFactors = FALSE
  )
}

In [44]:
unique_items <- level9_sales %>%
  distinct(store_id, dept_id, cat_id)

cl <- makeCluster(min(4, detectCores() - 1))
registerDoParallel(cl)

level9_forecasts <- foreach(
  i = 1:nrow(unique_items),
  .combine = rbind,
  .packages = c("dplyr", "forecast")
) %dopar% {
  item <- unique_items[i, ]
  item_data <- level9_sales %>%
    filter(store_id == item$store_id,
           dept_id == item$dept_id) %>%
    arrange(date)
  
  if (nrow(item_data) < 56) return(NULL)
  
  forecast_state_item(
    item_data,
    store_id = item$store_id,
    dept_id = item$dept_id,
    cat_id = item$cat_id
  )
}

stopCluster(cl)

In [45]:
# Convert to submission format
submission <- level9_forecasts %>%
  mutate(day = paste0("F", rep(1:28, length.out = nrow(level9_forecasts)))) %>%
  select(store_id, dept_id, day, forecast) %>%
  pivot_wider(names_from = day, values_from = forecast)

# Save to CSV
write.csv(submission, "submission_arimax9.csv", row.names = FALSE)

In [54]:
sales_out <- read.csv("sales_test_validation.csv", stringsAsFactors = FALSE)
stat_total <- read.csv("stat_total.csv", stringsAsFactors = FALSE)
forecasts <- read.csv("submission_arimax9.csv", stringsAsFactors = FALSE)  # Your forecast file

In [55]:
# Convert sales_out to long format
actuals_long <- sales_out %>%
  pivot_longer(
    cols = starts_with("d_"),
    names_to = "day",
    values_to = "actual"
  ) %>%
  mutate(
    day_num = as.numeric(gsub("d_", "", day)),
    series_id = paste(store_id, dept_id, sep = "_")
  )

# Convert forecasts to long format
forecasts_long <- forecasts %>%
  pivot_longer(
    cols = starts_with("F"),
    names_to = "day",
    values_to = "forecast"
  ) %>%
  mutate(
    day_num = as.numeric(gsub("F", "", day)) + 1913,
    series_id = paste(store_id, dept_id, sep = "_")
  )

In [56]:
weights <- stat_total %>%
  mutate(
    series_id = paste(store_id, dept_id, sep = "_"),
    weight = dollar_sales/sum(dollar_sales)  # Normalize to create weights
  ) %>%
  select(series_id, weight)

In [57]:
results <- actuals_long %>%
  # Clean column names first
  select(store_id, dept_id, day_num, actual, series_id) %>%
  inner_join(
    forecasts_long %>% 
      select(store_id, dept_id, day_num, forecast, series_id),
    by = c("series_id", "day_num"),
    suffix = c("", ".y")
  ) %>%
  # Calculate scaling factors
  group_by(series_id) %>%
  mutate(
    scale = mean(abs(diff(actual))),  # Scaling factor per series
    scaled_error = (forecast - actual)/scale
  ) %>%
  ungroup()

In [58]:
avg_scale <- mean(results$scale, na.rm = TRUE)
results <- results %>%
  mutate(scale = ifelse(scale == 0 | is.na(scale), avg_scale * 0.01, scale))

In [59]:
results <- results %>%
  mutate(scaled_error = ifelse(is.infinite(scaled_error), 0, scaled_error))

In [60]:
wrmsse <- results %>%
  left_join(weights, by = "series_id") %>%
  group_by(series_id) %>%
  summarise(
    rmse = sqrt(mean(scaled_error^2, na.rm = TRUE)),
    weighted_rmse = rmse * first(weight)
  ) %>%
  summarise(
    WRMSSE = sum(weighted_rmse, na.rm = TRUE)
  ) %>%
  pull(WRMSSE)

print(paste("Final WRMSSE:", round(wrmsse, 4)))

Warning message in left_join(., weights, by = "series_id"):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 1 of `x` matches multiple rows in `y`.
ℹ Row 1 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”


[1] "Final WRMSSE: 0.5404"
